In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time

#Random forest
from sklearn.ensemble import RandomForestClassifier

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV, StratifiedKFold


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score,roc_curve
from sklearn.base import BaseEstimator, TransformerMixin, clone

from scipy.stats import ttest_rel
from scipy.stats import randint, uniform

#hiperparamentros search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import randint, uniform,loguniform


# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_diab import *
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
start_inicial = time.time()


#Processo iniciado em: 10:52:55


## 1. Load Data & Pipeline

In [3]:
BASE = Path.cwd().parent

PP2 = joblib.load(BASE/'src'/'preprocess_diabetes_v1.2.joblib')['preprocessador']

DATA_DIR = BASE/"data"/"raw"
X_train = pd.read_csv(DATA_DIR/"X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR/"X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR/"y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR/"y_test_raw.csv").values.ravel()
mtd_scoring='roc_auc'

## 2.Baseline

In [3]:
def  best_threshold(y_test,yprob):
    # Threshold search
    thresholds = np.linspace(0.3, 0.7, 41)
    best_val = 0.5
    max_acc = 0
    for t in thresholds:
        acc = accuracy_score(y_test, yprob > t)
        if acc > max_acc:
            max_acc = acc
            best_val = t
    return best_val,max_acc

def print_hiper(search_1_best_params):
    
    print("🎯 Melhores Hiperparâmetros")
    print("-" * 50)
    for param, value in search_1_best_params.items():
        param_name = param.replace('model__', '').replace('_', ' ').title()
        print(f"• {param_name:<25} : {value}")
    print("-" * 50)

#=====================================================================
# model Baseline
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))

model_base = RandomForestClassifier(random_state=42, n_jobs=4)

pipe_base  = pipe_models(model_base, PP2) # Pipiline base ( PP2 é o preprocessador)

pipe_base  = pipe_models(model_base, PP2)

cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

pipe_base.fit(X_train, y_train)
y_probs0 = pipe_base.predict_proba(X_test)[:, 1]

threshold_0,max_acc_0=best_threshold(y_test,y_probs0)
auc_score_0 = roc_auc_score(y_test, y_probs0)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_0:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_0:.4f}")
print(f"⭐ AUC Score                : {auc_score_0:.4f}")
print(f"{'='*70}")
print_hiper(model_base.get_params())

print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


#Processo iniciado em: 17:51:06
🎯 Melhor Threshold         : 0.490
📈 Melhor Acurácia de Teste : 0.6630
⭐ AUC Score                : 0.6919
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Bootstrap                 : True
• Ccp Alpha                 : 0.0
• Class Weight              : None
• Criterion                 : gini
• Max Depth                 : None
• Max Features              : sqrt
• Max Leaf Nodes            : None
• Max Samples               : None
• Min Impurity Decrease     : 0.0
• Min Samples Leaf          : 1
• Min Samples Split         : 2
• Min Weight Fraction Leaf  : 0.0
• Monotonic Cst             : None
• N Estimators              : 100
• N Jobs                    : 4
• Oob Score                 : False
• Random State              : 42
• Verbose                   : 0
• Warm Start                : False
--------------------------------------------------

#Processo finalizado em: 17:51:39


## 3.Buscas por hiperparamentros


In [4]:
print("#Processo Iniciado em:", time.strftime("%H:%M:%S"))
# Exploratório
param_dist_1 = {
    # 1. Capacidade do Ensemble
    'model__n_estimators': randint(100, 3000),
    
    # 2. Complexidade Estrutural da Árvore(Profundidade e tamanho efetivo)
    'model__max_depth': [None, 2, 4, 8, 12, 15,20,25,30],
    'model__max_leaf_nodes': [None, 50, 100, 300, 800, 1500],

    # 3. Critérios de Divisão (Granularidade)
    'model__min_samples_split': randint(2, 40),
    'model__min_samples_leaf': randint(1, 30),
    'model__min_weight_fraction_leaf': uniform(0.0, 0.5),

    # 4. Estocasticidade e Diversidade do Ensemble
    # Equivalente conceitual ao subsample/colsample
    'model__max_features': ['sqrt', 'log2', None, 0.3, 0.5, 0.7],
    'model__bootstrap': [True, False],
    'model__max_samples': [None, 0.5, 0.7, 0.8, 0.9],

    # 5. Critério de Impureza (Bias-inducing)
    'model__criterion': ['gini', 'entropy']
}

cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
numero_itera=50


search_1 = RandomizedSearchCV(
    pipe_base, param_dist_1,
    n_iter=numero_itera, cv=cv_s,
    scoring=mtd_scoring,
    random_state=42, verbose=3
)

start = time.time()
search_1.fit(X_train, y_train)
end = time.time()

best_1 = search_1.best_estimator_

y_probs1 = best_1.predict_proba(X_test)[:, 1]

threshold_1,max_acc_1=best_threshold(y_test,y_probs1)
auc_score_1 = roc_auc_score(y_test, y_probs1)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_1:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_1:.4f}")
print(f"⭐ AUC Score                : {auc_score_1:.4f}")
print(f"{'='*70}")
print_hiper(search_1.best_params_)

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))

#Processo Iniciado em: 17:51:39
Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV 1/3] END model__bootstrap=True, model__criterion=entropy, model__max_depth=25, model__max_features=0.5, model__max_leaf_nodes=800, model__max_samples=0.5, model__min_samples_leaf=19, model__min_samples_split=24, model__min_weight_fraction_leaf=0.02904180608409973, model__n_estimators=2235;, score=0.693 total time= 4.1min
[CV 2/3] END model__bootstrap=True, model__criterion=entropy, model__max_depth=25, model__max_features=0.5, model__max_leaf_nodes=800, model__max_samples=0.5, model__min_samples_leaf=19, model__min_samples_split=24, model__min_weight_fraction_leaf=0.02904180608409973, model__n_estimators=2235;, score=0.690 total time= 3.7min
[CV 3/3] END model__bootstrap=True, model__criterion=entropy, model__max_depth=25, model__max_features=0.5, model__max_leaf_nodes=800, model__max_samples=0.5, model__min_samples_leaf=19, model__min_samples_split=24, model__min_weight_fraction_leaf=0.02

In [5]:
print("#Processo Iniciado em:", time.strftime("%H:%M:%S"))
def generate_refined_grid(best_params):
    """
    Gera um grid refinado automaticamente baseado nos melhores parâmetros
    do Random Search exploratório (Random Forest).
    """
    # ---------------------------
    # 1. n_estimators
    # ---------------------------
    n_best = best_params['model__n_estimators']
    n_min  = int(n_best * 0.8)
    n_max  = int(n_best * 1.2)
    # ---------------------------
    # 2. max_depth (Tratamento de None)
    # ---------------------------
    depth_best = best_params['model__max_depth']
    if depth_best is not None:
        depth_list = [
            max(1, depth_best - 2),
            depth_best,
            depth_best + 2
        ]
    else:
        depth_list = [None, 10, 20]

    # ---------------------------
    # 3. max_leaf_nodes (Tratamento de None)
    # ---------------------------
    leaf_best = best_params.get('model__max_leaf_nodes', None)
    if leaf_best is not None:
        leaf_list = [
            max(10, int(leaf_best * 0.8)),
            leaf_best,
            int(leaf_best * 1.2)
        ]
    else:
        leaf_list = [None, 100, 500]

    # ---------------------------
    # 4. min_samples_split
    # ---------------------------
    mss_best = best_params['model__min_samples_split']
    mss_list = [
        max(2, mss_best - 2),
        mss_best,
        mss_best + 2
    ]

    # ---------------------------
    # 5. min_samples_leaf
    # ---------------------------
    msl_best = best_params['model__min_samples_leaf']
    msl_list = [
        max(1, msl_best - 2),
        msl_best,
        msl_best + 2
    ]

    # ---------------------------
    # 6. min_weight_fraction_leaf
    # ---------------------------
    mwfl_best = best_params.get('model__min_weight_fraction_leaf', 0.0)
    mwfl_st   = max(0.0, mwfl_best * 0.75)
    mwfl_dist = 0.15

    # ---------------------------
    # 7. max_features
    # ---------------------------
    mf_best = best_params['model__max_features']
    if isinstance(mf_best, float):
        mf_list = [
            max(0.1, mf_best - 0.1),
            mf_best,
            min(1.0, mf_best + 0.1)
        ]
    else:
        mf_list = [mf_best]

    # ---------------------------
    # 8. max_samples (Tratamento de None)
    # ---------------------------
    ms_best = best_params.get('model__max_samples', None)
    if ms_best is not None:
        ms_st   = max(0.3, ms_best * 0.8)
        ms_dist = 0.25
    else:
        ms_st   = 0.5
        ms_dist = 0.3

    # ---------------------------
    # PRINTS DIAGNÓSTICOS
    # ---------------------------
    print(f"{'='*30} NOVO GRID DE REFINAMENTO (Random Forest) {'='*30}")
    print(f"🔹 n_estimators              : {n_min} - {n_max}")
    print(f"🔹 max_depth                 : {depth_list}")
    print(f"🔹 max_leaf_nodes            : {leaf_list}")
    print(f"🔹 min_samples_split         : {mss_list}")
    print(f"🔹 min_samples_leaf          : {msl_list}")
    print(f"🔹 min_weight_fraction_leaf  : {mwfl_st:.3f} - {mwfl_st + mwfl_dist:.3f}")
    print(f"🔹 max_features              : {mf_list}")
    print(f"🔹 max_samples               : {ms_st:.3f} - {min(1.0, ms_st + ms_dist):.3f}")
    print(f"{'='*92}\n")

    # ---------------------------
    # GRID FINAL
    # ---------------------------
    refined_grid = {
        'model__n_estimators': randint(n_min, n_max + 1),
        'model__max_depth': depth_list,
        'model__max_leaf_nodes': leaf_list,
        'model__min_samples_split': mss_list,
        'model__min_samples_leaf': msl_list,
        'model__min_weight_fraction_leaf': uniform(mwfl_st, mwfl_dist),
        'model__max_features': mf_list,
        'model__max_samples': uniform(ms_st, ms_dist)
    }

    return refined_grid

# # Aplicação automática
param_dist_2 = generate_refined_grid(search_1.best_params_)

numero_itera = 30
search_2 = RandomizedSearchCV(
    pipe_base, param_dist_2,
    n_iter=numero_itera, cv=cv_s, 
    scoring=mtd_scoring,
    random_state=42, verbose=1
)

start = time.time()
search_2.fit(X_train, y_train)
end = time.time()

best_2 = search_2.best_estimator_
y_probs2 = best_2.predict_proba(X_test)[:, 1]

threshold_2,max_acc_2=best_threshold(y_test,y_probs2)
auc_score_2 = roc_auc_score(y_test, y_probs2)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_2:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_2:.4f}")
print(f"⭐ AUC Score                : {auc_score_2:.4f}")
print(f"{'='*70}")
print_hiper(search_2.best_params_)

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


#Processo Iniciado em: 20:18:42
============================== NOVO GRID DE REFINAMENTO (Random Forest) ==============================
🔹 n_estimators              : 2096 - 3145
🔹 max_depth                 : [10, 12, 14]
🔹 max_leaf_nodes            : [1200, 1500, 1800]
🔹 min_samples_split         : [11, 13, 15]
🔹 min_samples_leaf          : [3, 5, 7]
🔹 min_weight_fraction_leaf  : 0.006 - 0.156
🔹 max_features              : ['sqrt']
🔹 max_samples               : 0.640 - 0.890

Fitting 3 folds for each of 30 candidates, totalling 90 fits
🎯 Melhor Threshold         : 0.550
📈 Melhor Acurácia de Teste : 0.6641
⭐ AUC Score                : 0.6955
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Max Depth                 : 14
• Max Features              : sqrt
• Max Leaf Nodes            : 1800
• Max Samples               : 0.7548122229914669
• Min Samples Leaf          : 3
• Min Samples Split         : 15
• Min Weight Fraction Leaf  : 0.009523087331276705
• N Es

In [6]:
joblib.dump(search_1.best_estimator_, 'modelo_RF_final_randsearch.'+mtd_scoring+'_v1.2.joblib')
joblib.dump(search_2.best_estimator_, 'modelo_RF_final_refine.'+mtd_scoring+'_v1.2.joblib')
print("\n#Arquivos salvos", time.strftime("%H:%M:%S"))



#Arquivos salvos 22:33:31
